- 1. 互相关运算

输出大小等于输入大小 $n_k * n_w$ 减去卷积核大小 $k_h * k_w$ 即:
$(n_h * k_h + 1) * (n_w - k_w + 1)$

In [102]:
import torch

#@save
def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()

    return Y

X = torch.tensor([[0,1,2], [3,4,5], [6,7,8]])
K = torch.tensor([[0,1], [2,3]])
print(corr2d(X, K))

tensor([[19., 25.],
        [37., 43.]])


In [123]:
def conv2d_by_mul(X, K):
    h, w = K.shape
    outh = X.shape[0] - h + 1
    outw = X.shape[1] - w + 1
    K = K.reshape(-1, 1)
    # print('outh', outh, 'outw', outw, 'K.shape ', K.shape)
    # print('X ', X)
    Y = []   # 张量列表
    for i in range(outh):
        for j in range(outw):
            # print(X[i:i + h, j:j + w].reshape(-1))
            Y.append(X[i:i + h, j:j + w].reshape(-1)) # 滑动窗口模式, 窗口里面的元素
    print(Y)
    Y = torch.stack(Y, 0)
    return torch.matmul(Y, K)

conv2d_by_mul(X, K)
# new_Y = conv2d_by_mul(X, K)
# new_Y_2 = torch.stack(new_Y, 0)
# print(new_Y)
# print(new_Y_2)

[tensor([0, 1, 3, 4]), tensor([1, 2, 4, 5]), tensor([3, 4, 6, 7]), tensor([4, 5, 7, 8])]


tensor([[19],
        [25],
        [37],
        [43]])

- 练习 如何通过改变输入张量和卷积核张量，将互相关运算表示为矩阵乘法？

In [133]:
def conv2d_by_mul(X, K):
    h, w = K.shape
    outh = X.shape[0] - h + 1
    outw = X.shape[1] - w + 1
    K = K.reshape(-1, 1)

    Y = torch.zeros((outh, outw))
    # 张量
    for i in range(outh):
        for j in range(outw):
            # print(X[i:i + h, j:j + w].reshape(-1))
            tensor_X = X[i:i + h, j:j + w].reshape(-1)
            slice_tensor_X = [tensor_X]
            # print(tensor_X)
            # print(slice_tensor_X)
            new_Y = torch.stack(slice_tensor_X, 0)
            # print(new_Y.shape)
            # print(K.shape)

            Y[i, j] = torch.matmul(new_Y, K)

    return Y

conv2d_by_mul(X, K)
# new_Y = conv2d_by_mul(X, K)
# new_Y_2 = torch.stack(new_Y, 0)
# print(new_Y)
# print(new_Y_2)

tensor([[19., 25.],
        [37., 43.]])

- 2. 卷积层

高度和宽度分别为 $h$ 和 $w$ 的卷积核可以被称为 $h*w$ 卷积或 $h*w$ 卷积核。 我们也将带有 $h*w$ 卷积核的卷积层称为 $h*w$ 卷积层

In [23]:
from torch import nn

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zero(1))

    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

- 3. 图像中目标的边缘检测

In [24]:
# X = torch.ones((6, 8))
X = torch.ones(6, 8)
X[:, 2:6] = 0
print(X)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


- 构造一个高度为 1、宽度为 2的卷积核K

当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零

In [25]:
K = torch.tensor([[1.0, -1.0]])
# print(K)

- 参数X（输入）和K（卷积核）执行互相关运算。 如下所示，输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘，其他情况的输出为0


In [26]:
Y = corr2d(X, K)
print(Y)

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


- 将输入的二维图像转置，再进行如上的互相关运算。 其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个卷积核K只可以检测垂直边缘，无法检测水平边缘

In [27]:
print(corr2d(X.t(), K))

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


- 4. 学习卷积核

In [96]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = torch.ones(6, 8)
X[:, 2:6] = 0
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)

X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)

lr = 3e-2 # 学习率, 调整这个参数为啥, loss 变化那么大

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i+1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

print(conv2d.weight.data.reshape((1, 2)))

epoch 2, loss 641.640
epoch 4, loss 170929.484
epoch 6, loss 45534684.000
epoch 8, loss 12130191360.000
epoch 10, loss 3231417892864.000
tensor([[560303.5000, 560303.4375]])


- 构建一个具有对角线边缘的矩阵
    - 如果将本节中举例的卷积核K应用于X，会发生什么情况？
    - 如果转置X会发生什么？
    - 如果转置K会发生什么？

In [89]:
import numpy as np

n = 4
X_main_diag = np.eye(n) # X_custom_main = np.diag(np.full(n, 5))
print(X_main_diag)

X_main_diag_tensor = torch.tensor(X_main_diag)
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X_main_diag_tensor, K)
print(Y)

print(corr2d(X_main_diag_tensor.t(), K)) # 转置 X
print(corr2d(X_main_diag_tensor, K.t())) # 转置 K

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
tensor([[ 1.,  0.,  0.],
        [-1.,  1.,  0.],
        [ 0., -1.,  1.],
        [ 0.,  0., -1.]])
tensor([[ 1.,  0.,  0.],
        [-1.,  1.,  0.],
        [ 0., -1.,  1.],
        [ 0.,  0., -1.]])
tensor([[ 1., -1.,  0.,  0.],
        [ 0.,  1., -1.,  0.],
        [ 0.,  0.,  1., -1.]])
